# Setup

In [1]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/pednet_design/code


In [2]:
from mobilkit.umni import *
from setup import P, City, Pednet

In [3]:
import osmnx
from pqdm.processes import pqdm
from shapely.geometry import LineString, Point
from sklearn.neighbors import KDTree

In [4]:
aus = City('Austin, TX')
cam = City('Cambridge, MA')
tor = City('Toronto, Ontario')

---
# Boundaries

## City boundary

In [5]:
def save_osm_city_boundary(city):
    city.save('boundary', osmnx.geocode_to_gdf(city.geocode)[['geometry']].to_crs(CRS_DEG))

**AUSTIN**

In [6]:
save_osm_city_boundary(aus)

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Boundary/BOUNDARY_CityBoundary)

In [7]:
save_osm_city_boundary(cam)

[**TORONTO**](https://open.toronto.ca/dataset/regional-municipal-boundary/)

Custom download

In [8]:
tor.save('boundary', gpd.read_file(tor.root / 'raw/toronto-boundary-wgs84.zip')[['geometry']].to_crs(CRS_DEG))
# save_osm_city_boundary(tor, 'Toronto, Ontario') # Not proper boundary

## Roads
Custom download and renamed to `{city.data}/raw/roads.zip`.

[**AUSTIN**](https://data.austintexas.gov/Locations-and-Maps/Street-Centerline/m5w3-uea6)

In [9]:
# t=0:10
df = (gpd.read_file(aus.root / 'raw/roads.zip')
      .rename(columns={'street_typ': 'kind', 'speed_limi': 'speed_limit'})
      [['kind', 'speed_limit', 'geometry']].rename_axis('id').to_crs(CRS_DEG))
aus.save('roads', df)

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_Centerlines)

In [10]:
df = (gpd.read_file(cam.root / 'raw/roads.zip')
      .rename(columns={'Direction': 'direction', 'MajorRoad': 'major_road'})
      [['direction', 'major_road', 'geometry']].rename_axis('id').to_crs(CRS_DEG))
cam.save('roads', df)

[**TORONTO**](https://open.toronto.ca/dataset/toronto-centreline-tcl/)

In [11]:
# t=0:10
df = (gpd.read_file(tor.root / 'raw/roads.zip')
      .rename(columns={'ONEWAY_DIR': 'one_way', 'FEATURE_00': 'kind'})
      [['kind', 'one_way', 'geometry']].rename_axis('id').to_crs(CRS_DEG))
tor.save('roads', df)

## Sidewalks

[**AUSTIN**](https://data.austintexas.gov/Locations-and-Maps/Sidewalks/pc5y-5bpw)

In [12]:
# t=0:38
df = (gpd.read_file(aus.root / 'raw/sidewalks.zip')
      .explode(index_parts=True).reset_index()
      .rename(columns={'functional': 'condition', 'pedestrian': 'kind'})
      .assign(exists = lambda df: df['kind'].isin(
          ['EXISTING_SIDEWALK', 'DRIVEWAY', 'SHARED_USE_PATH']))
      [['kind', 'exists', 'condition', 'geometry']]
      .rename_axis('id').to_crs(CRS_DEG))
aus.save('sidewalks', df)

In [13]:
aus.load('sidewalks').kind.value_counts().sum()

343773

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_SidewalkCenterlines)

In [14]:
df = (gpd.read_file(cam.root / 'raw/pednet.zip')
      .query('TYPE != "CWALK-CL"').reset_index()
      .rename(columns={'TYPE': 'kind'})
      [['kind', 'geometry']].rename_axis('id').to_crs(CRS_DEG))
cam.save('sidewalks', df)

[**TORONTO**](https://open.toronto.ca/dataset/pedestrian-network/)

In [15]:
# t=0:07
df = (gpd.read_file(tor.root / 'raw/pednet.zip')
      .query('crosswalk == 0').reset_index()
      .rename(columns={'sdwlk_desc': 'kind'})
      [['road_type', 'kind', 'geometry']]
      .assign(exists = lambda df: df['kind'].isin([
          'Sidewalk on both sides', 'City walkway',
          'Sidewalk on north side only', 'Sidewalk on south side only', 
          'Sidewalk on east side only', 'Sidewalk on west side only',
          'Sidewalk on east side; partially on other side',
          'Sidewalk on west side; partially on other side', 'Recreational Trail',
          'Laneway with sidewalk on west side',
          'Laneway with sidewalk on east side',
          'Sidewalk on north side; partially on other side',
          'Sidewalk on south side; partially on other side']))
      .rename_axis('id').to_crs(CRS_DEG))
tor.save('sidewalks', df)

## Crosswalks

[**AUSTIN**](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION-markings_short_line/3p2i-pqdc)

In [16]:
# t=0:02
df = (gpd.read_file(aus.root / 'raw/crosswalks.zip')
      .query('short_line == "CROSSWALK"')
      .explode(index_parts=True).reset_index()
      .assign(signalized = lambda df: df['signal_int'] == 'Y')
      .rename(columns={'crew_assig': 'location', 'marking_si': 'size'})
      [['location', 'size', 'signalized', 'geometry']]
     .reset_index(drop=True).rename_axis('id').to_crs(CRS_DEG))
aus.save('original_crosswalks', df)

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_SidewalkCenterlines)

In [17]:
df = (gpd.read_file(cam.root / 'raw/pednet.zip')
      .query('TYPE == "CWALK-CL"').reset_index()
      [['geometry']].rename_axis('id').to_crs(CRS_DEG))
cam.save('crosswalks', df)

[**TORONTO**](https://open.toronto.ca/dataset/pedestrian-network/)

In [18]:
# t=0:06
df = (gpd.read_file(tor.root / 'raw/pednet.zip')
      .query('crosswalk == 1').reset_index()
      .rename(columns={'cwalk_type':'kind'})
      [['road_type', 'kind', 'geometry']]
      .rename_axis('id').to_crs(CRS_DEG))
tor.save('crosswalks', df)

## Snap crosswalks to sidewalks

**AUSTIN**

In [19]:
# t=0:18
sw = aus.load('sidewalks')
xw = aus.load('original_crosswalks')
sw_pts = Pdf([(*x.coords[0], *x.coords[-1]) for x in sw.geometry], 
             columns=['x0','y0','x1','y1'])
sw_pt_tree = KDTree(np.unique(np.vstack([sw_pts[['x0','y0']].values, 
                                         sw_pts[['x1','y1']].values]), axis=0))
xw['source'] = list(zip(*Arr(sw_pt_tree.data)[sw_pt_tree.query(
    Arr([x.coords[0] for x in xw.geometry]))[1].flatten()].T))
xw['target'] = list(zip(*Arr(sw_pt_tree.data)[sw_pt_tree.query(
    Arr([x.coords[-1] for x in xw.geometry]))[1].flatten()].T))
xw['geometry'] = gpd.GeoSeries([
    LineString(x) for x in zip(xw.pop('source'), xw.pop('target'))], crs=CRS_DEG)
aus.save('crosswalks', xw)

---
# Candidate crosswalks

## Generate sidewalk endpoints

In [20]:
def make_candidate_xwalk_endpoints(city, buffer=20):
    """
    Generate a list of sidewalk endpoints within road intersections that will 
    then be used to generate pair-wise combinations.
    """
    # load base layers
    roads = (city.load('roads').set_crs(CRS_DEG).dropna(subset='geometry')
             .explode(index_parts=True).reset_index(drop=True).rename_axis('road_id'))
    sw = city.load('sidewalks').set_crs(CRS_DEG)
    xw = city.load('crosswalks').set_crs(CRS_DEG)
    # find intersections (as road endpoints)
    ixns = Pdf([dict(zip(['src','trg'], [x.coords[0], x.coords[-1]])) for x 
                in roads.geometry]).assign(road_id=roads.index)
    ixns = (ixns.melt('road_id').groupby('value')['road_id'].agg([list, 'count']).query('count > 1')
            .rename_axis('xy')['list'].rename('road_id').reset_index().rename_axis('ixn_id'))
    ixns = Gdf(ixns.assign(geometry=gpd.GeoSeries([Point(x) for x in ixns.pop('xy')], crs=CRS_DEG)))
    # create intersection buffers
    ixn_buf = ixns.assign(geometry=ixns.to_crs(CRS_M).buffer(buffer).to_crs(CRS_DEG))
    # sidewalk segments within intersection buffers
    sw_in = (gpd.sjoin(sw.rename_axis('sw_id'), ixn_buf.reset_index()).drop(
        columns='index_right').groupby('sw_id')['ixn_id'].agg(list).reset_index())
    # both endpoints of sidewalks in buffer
    df = sw.loc[sw_in['sw_id']].rename_axis('sw_id')
    src, trg = zip(*[(x.coords[0], x.coords[-1]) for x in df.geometry])
    df = pd.concat([Seq(src, index=df.index), Seq(trg, index=df.index)]).reset_index()
    df = df.groupby(0)['sw_id'].agg(list).rename_axis('xy').reset_index().rename_axis('sw_pt_id')
    sw_in_pts = Gdf(df.assign(geometry=gpd.GeoSeries([Point(x) for x in df.pop('xy')], crs=CRS_DEG)))
    # map sidewalk endpoint to intersection
    sw_pt2ixn = gpd.sjoin(sw_in_pts, ixn_buf.reset_index())['ixn_id'].reset_index()
    # sidewalk endpoints within intersections
    sw_pts_in = gpd.sjoin(sw_in_pts, ixn_buf).drop(columns='index_right')
    # assign sidewalks to nearest roads
    sw2road = (sw_in.explode('ixn_id').merge(ixns[['road_id']], on='ixn_id').explode('road_id')
               .merge(sw_in_pts.explode('sw_id')['sw_id'].reset_index(), on='sw_id'))
    sw2road['dist'] = (sw_in_pts.to_crs(CRS_M).loc[sw2road['sw_pt_id'], 'geometry'].distance(
        roads.to_crs(CRS_M).loc[sw2road['road_id'], 'geometry'], align=False).values)
    sw2road = (sw2road.groupby(['sw_id','road_id'])['dist'].mean().reset_index().sort_values('dist')
           .groupby('sw_id').head(1).reset_index()[['sw_id','road_id']])
    # get attributes of each candidate crosswalk endpoint (nearest road, swalk & intersection)
    xw_pts = (sw_in_pts.explode('sw_id')['sw_id'].reset_index()
              .merge(sw_in, on='sw_id')
              .merge(sw2road, on='sw_id').explode('ixn_id')
              .merge(ixns['road_id'].rename('rid2'), on='ixn_id').explode('rid2')
              .merge(sw_pts_in.reset_index()[['sw_pt_id']], on='sw_pt_id')
              .query('road_id == rid2').reset_index(drop=True)
              .merge(sw_pt2ixn.rename(columns={'ixn_id':'ixn_uid'}), on='sw_pt_id')
              .drop(columns='rid2').astype(np.int32))
    return sw_in_pts, xw_pts

sw_pts, candi_pts = make_candidate_xwalk_endpoints(cam)
candi_pts.disp(3); pass

9,745 rows x 5 cols; Memory: 0.3 MiB


,sw_pt_id,sw_id,ixn_id,road_id,ixn_uid
,<int32>,<int32>,<int32>,<int32>,<int32>
0,2,283,0,1302,0
1,3,1031,0,2575,0
2,4,1031,0,2575,0


## Create eligible endpoint combinations

In [21]:
def make_xwalk_combinations(grp_pair):
    ixn_id, df = grp_pair
    return (df.merge(df, how='cross').query(' and '.join([
        'sw_pt_id_x != sw_pt_id_y',
        'sw_id_x != sw_id_y',
        'road_id_x == road_id_y',
        'ixn_uid_x == ixn_uid_y'
    ])).rename(columns={'ixn_id_x': 'ixn_id', 'road_id_x': 'road_id'})
            .drop(columns=['ixn_id_y', 'ixn_uid_x', 'ixn_uid_y', 'road_id_y']))

In [22]:
def make_candidate_xwalk_segments(sw_pts, candi_pts, roads, xwalks):
    # generate crosswalk combinations from eligible sidewalk endpoints
    combs = (pd.concat(pqdm(candi_pts.groupby('ixn_id'), make_xwalk_combinations,
                            n_jobs=40, total=candi_pts['ixn_id'].nunique()))
             .reset_index(drop=True))
    # create candidate crosswalk linestrings from sidewalk endpoint pairs
    xw = (combs.merge(sw_pts['geometry'].rename('src').rename_axis('sw_pt_id_x').reset_index())
          .merge(sw_pts['geometry'].rename('trg').rename_axis('sw_pt_id_y').reset_index()))
    xw['geometry'] = gpd.GeoSeries([LineString(x) for x in zip(xw['src'], xw['trg'])])
    xw = Gdf(xw, crs=CRS_DEG).assign(**{c: xw[c].apply(lambda x: x.coords[0]) for c in ['src','trg']})
    # filter candidates that intersect their road and get their endpoints
    xw = xw[xw.geometry.intersects(roads.loc[xw['road_id'], 'geometry'], align=False)]
    # identify and remove existing crosswalks from the candidates
    xwalks = Pdf([dict(zip(['src','trg'], [x.coords[0], x.coords[-1]])) for x in xwalks.geometry])
    common_xw_ids = pd.concat([
        xw.reset_index().merge(xwalks, on=('src','trg'))['index'],
        xw.reset_index().merge(xwalks, left_on=('src','trg'), right_on=('trg','src'))['index']
    ]).drop_duplicates().values
    xw = xw.loc[list(set(xw.index) - set(common_xw_ids))].drop(columns=['src','trg']).rename_axis('id')
    # find the shortest crosswalk segment at each interesection leg
    xw['len'] = xw.to_crs(CRS_M).geometry.length
    xw = xw.loc[xw.sort_values('len').groupby(['ixn_id','road_id']).head(1).reset_index()['id']]
    return xw.reset_index(drop=True).rename_axis('id')
    
# %time make_candidate_xwalk_segments(sw_pts, candi_pts, cam.roads, cam.crosswalks).disp()

## Overall

In [23]:
def make_candidate_xwalks(city, save=True, overwrite=False):
    outfile = city.root / 'candidate_crosswalks.parquet'
    if outfile.exists() and not overwrite:
        return gpd.read_parquet(outfile)
    sw_pts, candi_pts = make_candidate_xwalk_endpoints(city)
    roads, cur_xwalks = city.load('roads'), city.load('crosswalks')
    candi = make_candidate_xwalk_segments(sw_pts, candi_pts, roads, cur_xwalks)
    if save:
        city.save('candidate_crosswalks', candi)
    return candi

**AUSTIN**

In [24]:
_ = make_candidate_xwalks(aus) # t=1:32

**CAMBRIDGE**

In [25]:
_ = make_candidate_xwalks(cam) # t=0:06

**TORONTO**

In [26]:
_ = make_candidate_xwalks(tor) # t=1:04

---
# Base pednet

In [27]:
def create_base_pednet(city, save=True, overwrite=False):
    outfiles = [city.root / f'full_pednet_{x}.parquet' for x in ['nodes', 'edges']]
    if all([f.exists() for f in outfiles]) and not overwrite:
        V = city.load('full_pednet_nodes')
        E = city.load('full_pednet_edges')
        return V, E
    sw = city.load('sidewalks').rename_axis('orig_id').assign(is_xwalk=False)
    if 'exists' not in sw.columns: sw['exists'] = True
    xw = city.load('crosswalks').rename_axis('orig_id').assign(is_xwalk=True, exists=True)
    candi = (city.load('candidate_crosswalks')[['geometry']].rename_axis('orig_id')
             .assign(is_xwalk=True, exists=False))
    E = pd.concat([sw.reset_index(), xw.reset_index(), candi.reset_index()])
    E['len'] = E.to_crs(CRS_M).geometry.length
    E = E[E['len'] > 0].reset_index(drop=True).rename_axis('eid').reset_index()
    E['src'], E['trg'] = zip(*[(x.coords[0], x.coords[-1]) for x in tqdm(E.geometry)])
    V = Seq(list(set(E['src']) | set(E['trg'])), name='xy').rename_axis('vid').reset_index()
    E = E.merge(V.rename(columns={'vid':'src_vid', 'xy':'src'}))
    E = E.merge(V.rename(columns={'vid':'trg_vid', 'xy':'trg'}))
    E = E.drop(columns=['src','trg']).set_index('eid').sort_index()
    E = E.astype({'src_vid': np.int32, 'trg_vid': np.int32, 'len': np.float32})
    V['geometry'] = gpd.GeoSeries([Point(x) for x in V['xy']], crs=CRS_DEG)
    V['x'], V['y'] = zip(*V['xy'])
    V = Gdf(V.set_index('vid'))[['x','y','geometry']]
    if save:
        city.save('full_pednet_nodes', V)
        city.save('full_pednet_edges', E)
    return V, E

# %time x = create_base_pednet(aus); x

**AUSTIN**

In [28]:
aus.V, aus.E = create_base_pednet(aus) # t=0:36

**CAMBRIDGE**

In [29]:
cam.V, cam.E = create_base_pednet(cam)

**TORONTO**

In [30]:
tor.V, tor.E = create_base_pednet(tor) # t=0:08